In [1]:
# ライブラリ
import requests
import datetime
import csv
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep

# アクセス先
url = 'https://diamond-rm.net/'

# データ保管リスト
item_list = []

# 新着記事一覧をダウンロード
response = requests.get(url)

# 新着記事一覧を解析
soup = BeautifulSoup(response.text, 'html.parser')

# データ抽出 => データ解析時に「div#plist1 div.e」のID属性「plist1」が消去されるのでCSSセレクタを変更
for elem in soup.select('main > div:nth-child(2) > div.e'):
    # 記事タイトル
    item_title = elem.select('a')[1].text
    # 記事本文へのURL
    item_url = elem.select('a')[1]['href']
    # 記事本文をダウンロード
    response = requests.get(item_url)
    # 記事本文を解析
    soup = BeautifulSoup(response.text, 'html.parser')
    # 記事発行日
    item_date = soup.select('div.pmeta.cf span')[0].text
    # カテゴリ
    item_category = soup.select('div.pmeta.cf ul.post-categories li:nth-child(1) a')[0].text
    # 記事本文を初期化
    item_detail = ''
    # 記事本文を統合
    for val in soup.select('div.wpeditor p'):
        item_detail += val.text
    # データ保管リストに登録
    item_list.append([item_title, item_url, item_date, item_category, item_detail])
    # 負荷軽減
    sleep(1)

# CSVファイル
csv_head = ['タイトル', 'URL', '発行日', 'カテゴリ', '本文']
csv_date = datetime.datetime.today().strftime('%Y%m%d%H')
csv_file = 'article_' + csv_date + '.csv'

# CSVファイルにデータ保存
with open(csv_file, 'w', errors='ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_head)
    writer.writerows(item_list)